In [ ]:
import nest_asyncio
from pydantic_ai import Agent, AgentRunResultEvent, AgentStreamEvent

from voice_agent_flow.llms import create_pydantic_azure_openai

nest_asyncio.apply()
import asyncio
from pprint import pprint

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(model)

**1. Message Types**

In [2]:
result_sync = agent.run_sync('What is the capital of Italy?')
print(result_sync.output)

The capital of Italy is **Rome**.


*Retrieve message history*   
1. A message is either a ModelRequest or a ModelResponse Message.   
2. ModelRequest and ModelResponse include a part parameter, which is a list of parts.   
3. Parts can be TextPart, ToolCallPart, ToolReturnPart, UserPromptPart and SystemPromptPart.   

In [3]:
result_sync.all_messages()

[ModelRequest(parts=[UserPromptPart(content='What is the capital of Italy?', timestamp=datetime.datetime(2026, 2, 21, 10, 16, 45, 903444, tzinfo=datetime.timezone.utc))], run_id='64dbfd38-2668-422c-a47c-d89de1647800'),
 ModelResponse(parts=[TextPart(content='The capital of Italy is **Rome**.')], usage=RequestUsage(input_tokens=14, output_tokens=10, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 21, 10, 16, 47, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-DBeKVBa3l4eTGYEuyL5fYYGLE3y4B', finish_reason='stop', run_id='64dbfd38-2668-422c-a47c-d89de1647800')]

**2. Stop on Structured Output Run**

In [6]:
from pydantic import BaseModel

class Person(BaseModel):
    """Defines a person"""
    name:str
    aget:str
    
events: list[AgentStreamEvent | AgentRunResultEvent] = []

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(
    model,
    output_type= str | Person,
    instructions = "You are a helpful assistant that answers questions about anything, but when the user metions a person, you will reply with a json object defined by the schema."

)


res = agent.run_sync("My name is Wang Huan, I am 32 years old, and I am a AI engineer living in Beijing.")
res

AgentRunResult(output=Person(name='Wang Huan', aget='32'))

In [7]:
res.all_messages()

[ModelRequest(parts=[UserPromptPart(content='My name is Wang Huan, I am 32 years old, and I am a AI engineer living in Beijing.', timestamp=datetime.datetime(2026, 2, 21, 10, 53, 51, 530826, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that answers questions about anything, but when the user metions a person, you will reply with a json object defined by the schema.', run_id='802401ca-5e4d-40e4-b0db-c51aa5ca3912'),
 ModelResponse(parts=[ToolCallPart(tool_name='final_result', args='{"name":"Wang Huan","aget":"32"}', tool_call_id='call_c9hodxOGVpujH6nNXLCKtZHN')], usage=RequestUsage(input_tokens=102, output_tokens=22, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 21, 10, 53, 52, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_detai

**Tool Call Message History**

In [11]:
def weather_tool(location: str) -> str:
    """Search the weather for a location."""
    return f"The weather in {location} is sunny."

agent = Agent(model, tools = [weather_tool])

res = agent.run_sync("What is the weather in Beijing today?")
res.all_messages()

[ModelRequest(parts=[UserPromptPart(content='What is the weather in Beijing today?', timestamp=datetime.datetime(2026, 2, 21, 11, 7, 20, 674836, tzinfo=datetime.timezone.utc))], run_id='d3ca3e60-3b74-460d-80ae-efab4c352617'),
 ModelResponse(parts=[ToolCallPart(tool_name='weather_tool', args='{"location":"Beijing"}', tool_call_id='call_NCdrxHzMY1BKTaKhu1H3SuAZ')], usage=RequestUsage(input_tokens=52, output_tokens=16, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 21, 11, 7, 21, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_details={'finish_reason': 'tool_calls'}, provider_response_id='chatcmpl-DBf7RgKCoYPORQTcBpkiIgcLQCczO', finish_reason='tool_call', run_id='d3ca3e60-3b74-460d-80ae-efab4c352617'),
 ModelRequest(parts=[ToolReturnPart(tool_name='weather_tool'

`ModelRequest(parts = [ToolCallPart(...), ...])` and `ModelResponse(parts = [ToolReturnPart(...), ...])` are emitted when a tool call is made and when the tool returns, respectively.

**Structured Output Streaming**

In [12]:
from pydantic import BaseModel

class Person(BaseModel):
    """Defines a person"""
    name:str
    aget:str
    
events: list[AgentStreamEvent | AgentRunResultEvent] = []

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(
    model,
    output_type= str | Person,
    instructions = "You are a helpful assistant that answers questions about anything, but when the user metions a person, you will reply with a json object defined by the schema."

)


async for event in agent.run_stream_events('My name is huan, 32 years old.'):
    events.append(event)
    print(event)


PartStartEvent(index=0, part=ToolCallPart(tool_name='final_result', args='', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn'))
FinalResultEvent(tool_name='final_result', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn')
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='{"', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='name', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='":"', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='hu', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='an', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='","', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='aget', tool_call_id='call_AgCyHALFWT5Q4mcBfhhZcECn'

**Text Response Streaming**

In [13]:
async for event in agent.run_stream_events('how are you? answe me in 2 words'):
    events.append(event)
    print(event)

PartStartEvent(index=0, part=TextPart(content='Doing'))
FinalResultEvent(tool_name=None, tool_call_id=None)
PartDeltaEvent(index=0, delta=TextPartDelta(content_delta=' well'))
PartDeltaEvent(index=0, delta=TextPartDelta(content_delta='.'))
PartEndEvent(index=0, part=TextPart(content='Doing well.'))
AgentRunResultEvent(result=AgentRunResult(output='Doing well.'))


**Tool Call Output Streaming**

In [14]:
def weather_tool(location: str) -> str:
    """Search the weather for a location."""
    return f"The weather in {location} is sunny."

agent = Agent(model, tools = [weather_tool])

async for event in agent.run_stream_events('What is the weather in Beijing today?'):
    events.append(event)
    print(event)

PartStartEvent(index=0, part=ToolCallPart(tool_name='weather_tool', args='', tool_call_id='call_mATiSoN3MSl4NepejxRwJUv8'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='{"', tool_call_id='call_mATiSoN3MSl4NepejxRwJUv8'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='location', tool_call_id='call_mATiSoN3MSl4NepejxRwJUv8'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='":"', tool_call_id='call_mATiSoN3MSl4NepejxRwJUv8'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='Be', tool_call_id='call_mATiSoN3MSl4NepejxRwJUv8'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='ijing', tool_call_id='call_mATiSoN3MSl4NepejxRwJUv8'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='"}', tool_call_id='call_mATiSoN3MSl4NepejxRwJUv8'))
PartEndEvent(index=0, part=ToolCallPart(tool_name='weather_tool', args='{"location":"Beijing"}', tool_call_id='call_mATiSoN3MSl4NepejxRwJUv8'))
FunctionToolCallEvent(part=ToolCallPart(tool_name=